In [41]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models
import numpy as np

In [42]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [51]:
model = TextEmbedding(model_name=model_handle)

query = "I just discovered the course. Can I join now?"

embedding = next(model.embed([query]))

print(min(embedding))

-0.11726373885183883


In [44]:
np.linalg.norm(embedding)

np.float64(1.0)

In [45]:
embedding.dot(embedding)

np.float64(1.0000000000000002)

In [52]:
query2 =  'Can I still join the course after the start date?'

In [53]:
embedding2 = next(model.embed([query2]))

In [50]:
embedding.dot(embedding2)

np.float64(0.9008528895674548)

In [55]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [59]:
embeddings = np.array(list(model.embed([doc['text'] for doc in documents])))

In [61]:
embeddings.shape

(5, 512)

In [62]:
embeddings.dot(embedding)

array([0.76296847, 0.81823782, 0.80853974, 0.7133079 , 0.73044992])

In [63]:
for doc in documents:
    doc["full_text"] = doc["question"] + " " + doc["text"]

In [64]:
full_text_embeddings = np.array(list(model.embed([doc['full_text'] for doc in documents])))

In [65]:
full_text_embeddings.dot(embedding)

array([0.85145432, 0.84365942, 0.8408287 , 0.7755158 , 0.80860078])

In [69]:
dims = []
for model in TextEmbedding.list_supported_models():
    dims.append(model["dim"])

In [70]:
min(dims)

384

In [71]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [72]:
client = QdrantClient("http://localhost:6333") 

In [80]:
EMBEDDING_DIMENSIONALITY = 384

In [81]:
# Define the collection name
client.delete_collection("zoomcamp-rag")  # Clean up if the collection already exists
collection_name = "zoomcamp-rag"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [83]:
model_handle = "BAAI/bge-small-en"

In [84]:
points = []
id = 0

for doc in documents:

    point = models.PointStruct(
        id=id,
        vector=models.Document(text=doc['question'] + ' ' + doc['text'], model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload={
            "text": doc['text'],
            "question": doc['question']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1

In [85]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [86]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [87]:
result = search(query)

In [88]:
result.points[0]

ScoredPoint(id=14, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'question': 'The course has already started. Can I still join it?'}, vector=None, shard_key=None, order_value=None)